In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys

project_module_path = os.path.abspath(os.path.join('../'))
if project_module_path not in sys.path:
    print(f'Adding {project_module_path} to system paths')
    sys.path.append(project_module_path)

# %env AWS_PROFILE=development

Adding /Users/bensoorajmohan/Development/ai-playground/pytorch-mnist-play/src to system paths


# Lighter Model

## Target
- Aggressively try to reduce the parameters
## Result
- Parameters: 7,538
- Accuracy:
| Accuracy | Best | Final |
| --- | --- | --- |
| Train | 99.28 | 99.28
| Test | 99.23 | 99.21
## Analysis
- Model is light enough
- Moderately overfitting (train acc > test acc)
- Accuracy has tanked


## Import all the libraries

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from tqdm import tqdm
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter

!pip install torchsummary
from torchsummary import summary

In [3]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

logging.info("logging configured")  # calling the root logger

INFO:root:logging configured


## Configure device

In [4]:
use_cuda = torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()

device: torch.device = None
if use_cuda:
    device = torch.device("cuda")
elif use_mps:
    device = torch.device("mps")
else:
    device = torch.device("cpu")

assert device is not None, "device must not be None"
assert isinstance(device, torch.device), "device must be an instance of torch.device"

## Download the dataset and configure loaders

In [5]:
torch.manual_seed(1)
BATCH_SIZE = 128

# Transforms
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train = datasets.MNIST('../../data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('../../data', train=False, download=True, transform=test_transforms)

dataloader_args = dict(shuffle=True, batch_size=BATCH_SIZE, num_workers=2, pin_memory=True) if (use_cuda or use_mps) else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)


## Model summary

In [6]:
from models import ModelV2
summary(ModelV2(), input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
              ReLU-2           [-1, 16, 28, 28]               0
            Conv2d-3           [-1, 16, 28, 28]           2,320
              ReLU-4           [-1, 16, 28, 28]               0
       BatchNorm2d-5           [-1, 16, 28, 28]              32
         MaxPool2d-6           [-1, 16, 14, 14]               0
            Conv2d-7            [-1, 8, 14, 14]           1,160
              ReLU-8            [-1, 8, 14, 14]               0
       BatchNorm2d-9            [-1, 8, 14, 14]              16
          Dropout-10            [-1, 8, 14, 14]               0
           Conv2d-11            [-1, 8, 14, 14]             584
             ReLU-12            [-1, 8, 14, 14]               0
      BatchNorm2d-13            [-1, 8, 14, 14]              16
        MaxPool2d-14              [-1, 

## Train and evaluate the model

### Configuration

In [9]:
from pytorch_mnist_play import Config, Trainer

summaryWriter=SummaryWriter("../../logs/mnist")
tConfig = Config(
    summaryWriter=summaryWriter,
    epochs=15
)
print(f"tConfig: {tConfig}")

modelV2 = ModelV2().to(device)

tConfig: batch_size=128 epochs=15 lr=0.01 momentum=0.9 seed=1 log_dir='logs/mnist' summaryWriter=<torch.utils.tensorboard.writer.SummaryWriter object at 0x16a848d90>


### Start the loop

In [10]:
trainer = Trainer(tConfig)
trainer.fit(modelV2, device, train_loader, test_loader, modelV2.name())

Epoch 1: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [00:06<00:00, 71.27it/s, batch_id=468, loss=0.0240]
INFO:pytorch_mnist_play.trainer:[TRAIN ModelV2] Epoch 01 - Accuracy: 55993/60000 (93.32%)
INFO:pytorch_mnist_play.trainer:[TEST ModelV2] Epoch 01 - Loss: 0.0563, Accuracy: 9825/10000 (98.25%)
Epoch 2: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [00:06<00:00, 73.52it/s, batch_id=468, loss=0.0129]
INFO:pytorch_mnist_play.trainer:[TRAIN ModelV2] Epoch 02 - Accuracy: 58879/60000 (98.13%)
INFO:pytorch_mnist_play.trainer:[TEST ModelV2] Epoch 02 - Loss: 0.0431, Accuracy: 9865/10000 (98.65%)
Epoch 3: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████